<h3>Analyze the literary work using Spark RDD</h3>

<h4>Вариант 3 - Алексей Николаевич Толстой – "Эмигранты"</h4>
<h6>https://github.com/mikhailGaluzin/bigData2laba.git</h6>

<h5>Импорт библиотеки findspark и запуск Spark</h5>

In [3]:
import findspark
findspark.init()

<h5>Импорт необходимых классов из библиотек Spark</h5>

In [19]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from nltk.stem import SnowballStemmer
from nltk.stem.snowball import RussianStemmer
from operator import add

<h5>Инициализация Spark с настройками приложения</h5>

In [5]:
conf = SparkConf().setAppName("bigData2") # Создание и настройка объекта конфигурации Spark
sc = SparkContext.getOrCreate(conf=conf) # Получение или создание Spark-контекста
spark = SparkSession(sc)

<h5>Загрузка текстового файла</h5>

In [6]:
text_file_path = "book.txt"
text = sc.textFile(text_file_path)

<br></br>
<h5>Удаление пунктуации и первод слов в нижний регистр</h5>

In [7]:
text.take(10)

['Алексей Николаевич Толстой',
 '',
 'Эмигранты',
 '',
 '',
 'Факты этой повести исторически подлинны, вплоть до имен участников стокгольмских убийств. Профессор Стокгольмского университета сообщил мне подробности этого забытого дела. Остальные персонажи и сцены взяты по возможности документально из материалов, из устных рассказов и личных наблюдений. В первой редакции эта повесть называлась «Черное золото».',
 '',
 '\xa0\xa0\xa0\xa0А. Толстой',
 '',
 '']

In [8]:
def clean_low_reg(x): # Функция очистки пунктуации и перевод слов в нижний регистр
    lowcase = x.lower() # Перевод в нижний регистр
    punctuation = '!,."$%^&*()_+[]{}:;«»/-–_`~'
    for i in punctuation:
        lowcase = lowcase.replace(i, '') # замена символов на пробел
    return(lowcase)
text = text.map(clean_low_reg)
text.take(10)

['алексей николаевич толстой',
 '',
 'эмигранты',
 '',
 '',
 'факты этой повести исторически подлинны вплоть до имен участников стокгольмских убийств профессор стокгольмского университета сообщил мне подробности этого забытого дела остальные персонажи и сцены взяты по возможности документально из материалов из устных рассказов и личных наблюдений в первой редакции эта повесть называлась черное золото',
 '',
 '\xa0\xa0\xa0\xa0а толстой',
 '',
 '']

<br></br>
<h5>Очистка текста от стоп-слов</h5>

<h6>Стоп-слова</h6>

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('russian'))
print(stopwords)

{'впрочем', 'можно', 'нет', 'совсем', 'уже', 'про', 'раз', 'ней', 'два', 'об', 'чуть', 'тем', 'больше', 'же', 'нибудь', 'такой', 'может', 'нельзя', 'себе', 'было', 'уж', 'за', 'о', 'без', 'три', 'есть', 'тот', 'этой', 'если', 'даже', 'была', 'не', 'вас', 'куда', 'эти', 'чем', 'из', 'мне', 'ним', 'их', 'почти', 'конечно', 'нас', 'она', 'что', 'как', 'этого', 'много', 'его', 'до', 'вам', 'чтобы', 'вы', 'ж', 'всю', 'ее', 'у', 'все', 'под', 'вдруг', 'над', 'тогда', 'всегда', 'никогда', 'сейчас', 'от', 'через', 'один', 'был', 'на', 'где', 'бы', 'то', 'когда', 'ничего', 'им', 'него', 'ведь', 'ну', 'хорошо', 'себя', 'какой', 'вот', 'эту', 'так', 'с', 'кто', 'ему', 'или', 'наконец', 'этот', 'быть', 'были', 'ей', 'ты', 'более', 'ни', 'свою', 'после', 'со', 'тут', 'по', 'они', 'потому', 'тебя', 'опять', 'чего', 'лучше', 'но', 'них', 'будто', 'мой', 'только', 'того', 'разве', 'и', 'при', 'другой', 'он', 'во', 'какая', 'еще', 'а', 'иногда', 'надо', 'сам', 'мы', 'будет', 'всего', 'тоже', 'потом', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\misch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<br></br>
<h6>Текст без стоп-слов</h6>

In [10]:
def remove_stopwords(sentence):
    words = sentence.split()
    remove_words = [word for word in words if word not in stopwords] # перебираем все слова и сравниваем со стоп-словами
    return ' '.join(remove_words)
text = text.map(remove_stopwords)
text.take(10)

['алексей николаевич толстой',
 '',
 'эмигранты',
 '',
 '',
 'факты повести исторически подлинны вплоть имен участников стокгольмских убийств профессор стокгольмского университета сообщил подробности забытого дела остальные персонажи сцены взяты возможности документально материалов устных рассказов личных наблюдений первой редакции эта повесть называлась черное золото',
 '',
 'толстой',
 '',
 '']

<br></br>
<h5>Удаление больших пробелов и абзацев в тексте</h5>

In [11]:
text = text.filter(lambda x: x!='')
text.take(8)

['алексей николаевич толстой',
 'эмигранты',
 'факты повести исторически подлинны вплоть имен участников стокгольмских убийств профессор стокгольмского университета сообщил подробности забытого дела остальные персонажи сцены взяты возможности документально материалов устных рассказов личных наблюдений первой редакции эта повесть называлась черное золото',
 'толстой',
 '1',
 'летом тысяча девятьсот девятнадцатого года ветер океана приносил короткие ливни солнце сквозь разрывы облаков освещало мокрые асфальты парижа бульвары каштановые аллеи аспидные крыши полосатые парусины столиками кабачков потоки потрепанных автомобилей снова вернувшихся полей войны услугам парижан иностранцев',
 'город испускал сложное благоухание центральные бульвары пахли бензином духами боковые улички ванилью овощами винными лавками непроветренными постелями гигантские железостеклянные рынки всеми дарами моря земли старых взбирающихся холмы извилистых улицах жили те чье мускульное напряжение наполняло город золот

<br></br>

<h5>Разделение предложений на слова</h5>

In [12]:
ready_text = text.flatMap(lambda x: x.split(" "))
ready_text.take(10)

['алексей',
 'николаевич',
 'толстой',
 'эмигранты',
 'факты',
 'повести',
 'исторически',
 'подлинны',
 'вплоть',
 'имен']

<br></br>
<h5>Топ-50 самых встречаемых слов в тексте<h5>

<p><b>reduceByKey</b> объединяет пары с одинаковыми ключами (словами) и применяет функцию add к значениям с одинаковыми ключами.<p><p><b>  ad </b>  складывает значени  для каждого уникального слова, таким образом, подсчитывая общее количество вхождени.p6>

In [13]:
word_count = ready_text.map(lambda word: (word, 1)).reduceByKey(add)
top_50_words = word_count.takeOrdered(50, key=lambda x: -x[1])
print(top_50_words)

[('это', 360), ('лаше', 273), ('хаджет', 188), ('бистрем', 161), ('сказал', 147), ('левант', 120), ('человек', 106), ('лицо', 104), ('вера', 103), ('налымов', 100), ('глаза', 96), ('леви', 89), ('руки', 79), ('лили', 79), ('юрьевна', 78), ('время', 74), ('очень', 68), ('нужно', 67), ('россии', 66), ('несколько', 60), ('лисовский', 60), ('господа', 59), ('левицкий', 59), ('дело', 57), ('ардашев', 56), ('тысяч', 55), ('нам', 53), ('мари', 53), ('николай', 52), ('глазами', 49), ('тебе', 49), ('руку', 47), ('генерал', 47), ('голову', 45), ('бистрема', 42), ('люди', 41), ('ответил', 41), ('деньги', 41), ('сегодня', 40), ('пять', 39), ('черт', 39), ('лиги', 39), ('дверь', 39), ('большевиков', 37), ('весь', 37), ('василий', 36), ('день', 36), ('спросил', 36), ('денисов', 36), ('манташев', 36)]


<br></br>
<h5>Топ-50 наименее распространенных слов в тексте</h5>

In [14]:
low_50_words = word_count.takeOrdered(50, key=lambda x: x[1])
print(low_50_words)

[('алексей', 1), ('николаевич', 1), ('факты', 1), ('участников', 1), ('университета', 1), ('персонажи', 1), ('материалов', 1), ('повесть', 1), ('называлась', 1), ('ливни', 1), ('разрывы', 1), ('асфальты', 1), ('каштановые', 1), ('парусины', 1), ('потрепанных', 1), ('испускал', 1), ('пахли', 1), ('лавками', 1), ('непроветренными', 1), ('железостеклянные', 1), ('взбирающихся', 1), ('извилистых', 1), ('чье', 1), ('жареной', 1), ('картошкой', 1), ('ацетиленовыми', 1), ('уличных', 1), ('палаток', 1), ('жарились', 1), ('вафли', 1), ('разгонял', 1), ('полутеней', 1), ('зеркальный', 1), ('асфальт', 1), ('каштановая', 1), ('лоснились', 1), ('проборы', 1), ('нездоровье', 1), ('загримированных', 1), ('послевоенной', 1), ('нехорошее', 1), ('юношей', 1), ('свинцовая', 1), ('разлагаться', 1), ('африканцев', 1), ('нагонял', 1), ('тление', 1), ('странные', 1), ('поражавшие', 1), ('комбинированными', 1)]


<br></br>
<h5>Stemming</h5>
<p>Приводим слова к основной форме для уменьшения размерности</p>

In [25]:
snowball = SnowballStemmer(language="russian")
stemmed_text = ready_text.map(lambda word: snowball.stem(word))
stemmed_text.take(30)

['алекс',
 'николаевич',
 'толст',
 'эмигрант',
 'факт',
 'повест',
 'историческ',
 'подлин',
 'вплот',
 'им',
 'участник',
 'стокгольмск',
 'убийств',
 'профессор',
 'стокгольмск',
 'университет',
 'сообщ',
 'подробн',
 'забыт',
 'дел',
 'остальн',
 'персонаж',
 'сцен',
 'взят',
 'возможн',
 'документальн',
 'материал',
 'устн',
 'рассказ',
 'личн']

<br></br>
<h5>Топ-50 самых встречаемых и наименнее встречаемых слов в тексте после стеминга</h5>

In [27]:
word_count_stem = stemmed_text.map(lambda word: (word, 1)).reduceByKey(add)
top_words = word_count_stem.takeOrdered(50, key=lambda x: -x[1])
low_words = word_count_stem.takeOrdered(50, key=lambda x: x[1])


<p>Встречаемые</p>

In [28]:
print(top_words)

[('эт', 462), ('лаш', 273), ('сказа', 208), ('рук', 203), ('лиц', 200), ('вер', 196), ('хаджет', 189), ('глаз', 188), ('бистр', 177), ('левант', 168), ('человек', 158), ('русск', 144), ('одн', 142), ('юрьевн', 130), ('дел', 128), ('сам', 127), ('ваш', 116), ('голов', 113), ('друг', 110), ('росс', 106), ('мо', 106), ('сво', 103), ('большевик', 101), ('налым', 100), ('лев', 100), ('говор', 99), ('нужн', 95), ('стол', 94), ('наш', 93), ('так', 92), ('бел', 92), ('дом', 92), ('левицк', 90), ('париж', 87), ('лисовск', 85), ('подня', 83), ('лил', 80), ('голос', 78), ('врем', 78), ('тысяч', 77), ('двер', 75), ('лиг', 75), ('котор', 73), ('стал', 72), ('революц', 72), ('генера', 70), ('ответ', 68), ('очен', 68), ('перв', 67), ('петроград', 67)]


<p></p>
<p>Не встречаемые</p>

In [30]:
print(low_words)

[('николаевич', 1), ('устн', 1), ('парусин', 1), ('благоухан', 1), ('ванил', 1), ('железостекля', 1), ('чье', 1), ('мускульн', 1), ('ацетиленов', 1), ('палаток', 1), ('вафл', 1), ('рулетк', 1), ('разгоня', 1), ('пленительн', 1), ('лазур', 1), ('загримирова', 1), ('разлага', 1), ('промежуточн', 1), ('нагоня', 1), ('карбункул', 1), ('нарыв', 1), ('флексотон', 1), ('мурлыка', 1), ('буха', 1), ('подошвами…', 1), ('побоищ', 1), ('перемеша', 1), ('субботн', 1), ('уйм', 1), ('завядш', 1), ('бутоньерк', 1), ('смокингов', 1), ('рубашек', 1), ('кровью?', 1), ('сглад', 1), ('стабилизац', 1), ('ракет', 1), ('потряст', 1), ('сексуальнокровав', 1), ('раул', 1), ('задуш', 1), ('сжег', 1), ('корот', 1), ('мистангет', 1), ('насеком', 1), ('нарывн', 1), ('медл', 1), ('заповед', 1), ('сонин', 1), ('япон', 1)]
